[View in Colaboratory](https://colab.research.google.com/github/sabumjung/Tensorflow/blob/master/QTable.ipynb)

In [2]:
!pip install gym


    100% |████████████████████████████████| 1.5MB 9.2MB/s 
    100% |████████████████████████████████| 1.0MB 13.3MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/cb/14/71/f4ab006b1e6ff75c2b54985c2f98d0644fffe9c1dddc670925
Successfully built gym


In [0]:
import gym
import tensorflow as tf
import numpy as np

In [0]:
#Open AI Gym 환경을 가져오기
env = gym.make('FrozenLake-v0')

In [5]:
#상/하/좌/우와 같은 discrete action에 대해 random 탐색 적용을 위한 값 설정(explore)
#아래와 같은 값에 대해 epoch별로 가중치를 설정한다.(초기에는 가중치를 높게, 완료단계에서는 가중치를 적게 가져감)
np.random.randn(1,env.action_space.n)

array([[-0.2497138 , -0.23980764,  0.35828463,  0.99537372]])

In [20]:
Q = np.zeros([env.observation_space.n,env.action_space.n])
s=env.reset()
print(s)
print(Q[s,:])
print(np.argmax(Q[s,:]))
print(np.argmax(Q[s,:])+np.random.randn(1,env.action_space.n))
print(1+np.random.randn(1,env.action_space.n))

0
[0. 0. 0. 0.]
0
[[ 0.90736319 -3.32656292 -0.21690185 -0.25214655]]
[[0.12772619 0.59298335 1.51936042 0.75731805]]


In [28]:
#Q테이블을 모두 0으로 초기화한다.
Q = np.zeros([env.observation_space.n,env.action_space.n])
#학습파라미터를 설정한다.
lr = .8
#시간에 따른 reward의 보상값 감소율을 결정한다.
y = .95
#총 epoch개수는 2000으로 설정한다.
num_episodes = 2000
#매 에피소드마다 보상(rewards)과 steps(단계)를 포함하고 있는 리스트를 생성한다.
rList = []
for i in range(num_episodes):
    #환경과 state값 초기화 : 0으로 설정
    s = env.reset()
    #rAll값 초기화 : 0으로 설정
    rAll = 0
    #d: 목표지점에 도달했는지 여부를 표시하는 값 초기화
    d = False
    
    #Q-Table 학습 알고리즘을 실행한다.(0~99까지)
    j = 0
    while j < 99:
        j+=1
        #Q테이블의 값을 대상으로 greedy 정책을 이용하여 Action을 선택한다.
        #랜덤변수를 이용하여 랜덤탐색을 한다. 랜덤변수에 대해 초기에는 가중치를 많이 설정(Explore)한다.
        #epoch이 진행될수록 action random값 설정에 대한 가중치를 적게 설정(exploit)한다.
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #환경으로부터 신규상태와 보상값을 가져온다.
        s1,r,d,_ = env.step(a)
        #새로운 지식으로 Q테이블의 값을 업데이트한다.
        Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])

        print(j, env.step(a))
        
        rAll += r
        s = s1
        if d == True:
            break
    rList.append(rAll)


1 (4, 0.0, False, {'prob': 0.3333333333333333})
2 (4, 0.0, False, {'prob': 0.3333333333333333})
3 (5, 0, True, {'prob': 1.0})
1 (0, 0.0, False, {'prob': 0.3333333333333333})
2 (5, 0.0, True, {'prob': 0.3333333333333333})
3 (5, 0, True, {'prob': 1.0})
1 (0, 0.0, False, {'prob': 0.3333333333333333})
2 (1, 0.0, False, {'prob': 0.3333333333333333})
3 (5, 0, True, {'prob': 1.0})
1 (0, 0.0, False, {'prob': 0.3333333333333333})
2 (8, 0.0, False, {'prob': 0.3333333333333333})
3 (5, 0.0, True, {'prob': 0.3333333333333333})
4 (5, 0, True, {'prob': 1.0})
1 (1, 0.0, False, {'prob': 0.3333333333333333})
2 (4, 0.0, False, {'prob': 0.3333333333333333})
3 (4, 0.0, False, {'prob': 0.3333333333333333})
4 (12, 0.0, True, {'prob': 0.3333333333333333})
5 (12, 0, True, {'prob': 1.0})
1 (0, 0.0, False, {'prob': 0.3333333333333333})
2 (0, 0.0, False, {'prob': 0.3333333333333333})
3 (5, 0.0, True, {'prob': 0.3333333333333333})
4 (5, 0, True, {'prob': 1.0})
1 (4, 0.0, False, {'prob': 0.3333333333333333})
2 (0, 

In [22]:
print("Score over time: " +  str(sum(rList)/num_episodes))
print("Final Q-Table Values")
print(Q)

Score over time: 0.67
Final Q-Table Values
[[2.28400629e-01 8.28311922e-03 9.04875021e-03 9.75700925e-03]
 [2.73335556e-03 3.55035572e-04 0.00000000e+00 1.73949509e-01]
 [3.06046672e-03 1.09476704e-01 3.85876254e-03 5.03410827e-03]
 [4.60506160e-04 1.77428570e-04 0.00000000e+00 1.07077404e-01]
 [4.04368348e-01 2.39861716e-03 1.26991357e-04 8.65551401e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.85118024e-05 2.97527945e-05 1.71350578e-01 3.26765263e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.03809917e-03 3.79138160e-03 5.53724930e-03 5.73644556e-01]
 [0.00000000e+00 8.13840540e-01 2.35019714e-03 0.00000000e+00]
 [7.86299202e-01 1.03194903e-03 3.20901286e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 8.69064562e-01 0.00000000e+00]
 [0.00000000e+00 9.74341766e-01 0.00000000e+00 0.00000000e+00]
 [0.00000000